# 5.3 Lab Cross Validation with Bootstrap

In [1]:
import numpy as np
import statsmodels.api as sm


In [2]:
pip install ISLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 13.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.

In [3]:
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)
from sklearn.model_selection import train_test_split

In [4]:
from functools import partial
from sklearn.model_selection import \
(cross_validate,
KFold,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

## 5.3.1 The Validation Set Approach

In [5]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size = 196, random_state = 0)

In [6]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [7]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
pred = results.predict(X_valid)
np.mean((y_valid - pred) ** 2)

23.61661706966988

In [8]:
def evalMSE(terms, response, train, test):
  mm = MS(terms)
  X_train = mm.fit_transform(train)
  y_train = train[response]
  X_test = mm.transform(test)
  y_test = test[response]
  results = sm.OLS(y_train, X_train).fit()
  test_pred = results.predict(X_test)
  return np.mean((y_test - test_pred)**2)


In [9]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
  MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)

MSE

array([23.61661707, 18.76303135, 18.79694163])

* Seems the square and cube perform better than a linear fit!

In [10]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
  MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

* As predicted from chapter 5 if we reconduct the models then we expect different answers since it is random.

* However the linear is still outperformed by the quadradic terms

## 5.3.2 Cross Validation

In [11]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

In [12]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
  X = np.power.outer(H, np.arange(d+1))
  M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
  cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])

In [13]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
  X = np.power.outer(H, np.arange(d+1))
  M_CV = cross_validate(M, X, Y, cv=cv)
  cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720065])

### Results - LOOCV , K -CV, Train/Test

* We see in all of the above models that the quadradic fit is much better than the linear fit

* Also see there is not much of a difference between the LOOCV and K- CV model.

* We see the error rate is slightly off from the train/test since it has a higher variance of variability unlike the other 2

In [14]:
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation);
results['test_score']

array([23.61661707])

In [15]:
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.4218450941091842)

### Results - Shuffle Split and Mean/STD

* We can see we can use K-fold method above or a shuffle split where we shuffle the splits up.

* Also can calculate the mean and std from our cross validated results

## 5.3.3 The Bootstrap

In [19]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
  cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
  return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [20]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [21]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

In [22]:
def boot_SE(func, D, n=None, B=1000, seed=0):
  rng = np.random.default_rng(seed)
  first_, second_ = 0, 0
  n = n or D.shape[0]
  for _ in range(B):
    idx = rng.choice(D.index, n, replace=True)
    value = func(D, idx)
    first_ += value
    second_ += value**2
  return np.sqrt(second_ / B - (first_ / B)**2)

In [23]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699

In [24]:
def boot_OLS(model_matrix, response, D, idx):
  D_ = D.loc[idx]
  Y_ = D_[response]
  X_ = clone(model_matrix).fit_transform(D_)
  return sm.OLS(Y_, X_).fit().params

In [25]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [28]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

KeyError: "None of [Index([333, 249, 200, 105, 120,  16,  29,   6,  68, 318,\n       ...\n       306, 206, 294, 205, 120,  34, 205, 384,  14, 223],\n      dtype='object', name='name', length=392)] are in the [index]"

In [29]:
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

In [30]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [31]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [32]:
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64